In [ ]:
# Created by: Adam Fabo
# Date: 22.5.2022
# Created at HMU Crete
# Class: Neural Networks
# File contains script to create first sample neural network (Chapter 3 in documentation)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import neurolab as nl
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
    
# change plot dimensions
plt.rcParams['figure.figsize'] = [8,2]
plt.rcParams['figure.dpi'] = 100

In [ ]:
# load data
data = pd.read_csv('data_banknote_auth_trimmed.txt', sep=",", header=None)

# data description
# 1. variance of Wavelet Transformed image (continuous)
# 2. skewness of Wavelet Transformed image (continuous)
# 3. curtosis of Wavelet Transformed image (continuous)
# 4. entropy of image (continuous)
# 5. class (integer)

data.columns = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]

data.head()



In [ ]:
# get class as separate array
target = data.copy()["Class"]
target = target.to_numpy()

# two categories so 2 neurons, change 0 to [0,1] an 1 to [1,0]
banknotes = {0: [0,1], 1: [1,0]}
target = [ banknotes[number] for number in target]

target = np.array(target)


data = data.drop(columns = ["Class"])
data = data.to_numpy()


In [ ]:
# split the dataset

X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.3)

# scale values to range (0,1)
min_max_scaler = preprocessing.MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train)
X_test  = min_max_scaler.fit_transform(X_test)


In [ ]:
# create NN with 4 neurons in hidden layer and 2 in output
net = nl.net.newff(nl.tool.minmax(X_train),[4,2])

net.layers[-1].transf = nl.trans.LogSig()
net.layers[ 0].transf = nl.trans.LogSig()


# set train function
net.trainf = nl.train.train_rprop



In [ ]:
# train the NN and get error
error = net.train(X_train,y_train,epochs = 3000, show = 100, lr = 0.05, goal=1e-5)


In [ ]:
plt.plot(error)
plt.title("Error of NN during training")
plt.xlabel("Epoch Number")
plt.ylabel("Train Error")
plt.grid()
#plt.savefig('images/First_NN_error.png',bbox_inches='tight')
plt.show()

In [ ]:

# simulate known set

out = net.sim(X_train)
out = np.around(out)

# calculate accuracy

correct = (out == y_train).all(axis = 1)
acc = (np.sum(correct)/len(out))*100

print("Accuracy of trained NN on known data is " +str(round(acc,2)) + "%")


In [ ]:

# simulate test set

out = net.sim(X_test)
out = np.around(out)

# calculate accuracy

correct = (out == y_test).all(axis = 1)
acc = (np.sum(correct)/len(out))*100

print("Accuracy of trained NN on unknow data is " +str(round(acc,2)) + "%")


In [ ]:
# reverse change output of NN [0,1] to 0 and [1,0] to 1
# [0,0] and [1,1] are not valid outputs but poorly trained NN can output those     

reverse_banknotes = {(0,1):0, (1,0):1, (0,0):2, (1,1):3}

# reverse output
out_reversed = [ reverse_banknotes[tuple(pair)] for pair in out]
out_reversed = np.array(out_reversed)

# reverse test set
y_test_reversed = [ reverse_banknotes[tuple(pair)] for pair in y_test]
y_test_reversed = np.array(y_test_reversed)

In [ ]:
# plot test and output

plt.plot(out_reversed,'r.')
plt.title('Output of NN')
plt.xlabel("Sample number")
plt.ylabel("Value")
plt.grid(True)
plt.savefig('images/First_NN_output_pattern.png',bbox_inches='tight')
plt.figure()


plt.plot(y_test_reversed,'r.')
plt.title('Expected output of NN')
plt.xlabel("Sample number")
plt.ylabel("Value")
plt.grid(True)
plt.savefig('images/First_NN_expected_pattern.png',bbox_inches='tight')
plt.show()

In [ ]:
# plot test and output only there where they differ 

# get boolean array of differences
bool_arr = (out_reversed != y_test_reversed)

# get indexes of differences
indexes = np.where(bool_arr == True)

# get values of differences
out_diff_vals    = out_reversed[bool_arr]
y_test_diff_vals = y_test_reversed[bool_arr]


# plot ddifferences
plt.plot(indexes[0],out_diff_vals,'ro',alpha = 0.5)
plt.plot(indexes[0],y_test_diff_vals,'bo',alpha = 0.5)
plt.title('Differences')
plt.xlabel("Sample number")
plt.ylabel("Sample value")
plt.legend(["Out", "Test"])
plt.grid(True)
#plt.savefig('images/First_NN_pattern_diff.png',bbox_inches='tight')
plt.show()

In [ ]:
len(error)

In [ ]:
error[-1]

In [ ]:
len(y_test)